# Stage 2: The Context-Engineered RAG

## Introduction

Welcome to the second stage of our Progressive Agents course. In Stage 1, we saw how a "naive" RAG agent suffered from Information Overload, wasting thousands of tokens and confusing the LLM with irrelevant data.

In this stage, we will apply the Context Engineering techniques we learned in Section 2 to fix these problems.

### Learning Objectives
In this lesson, you will:
1.  Implement Context Engineering: Apply cleaning, transformation, and optimization to your data.
2.  Measure the Impact: See a dramatic reduction in token usage (aiming for ~90% reduction).
3.  Understand the Trade-offs: Recognize that while we saved tokens, we lost some detail (like syllabi) by applying a "flat" strategy.

### The Scenario
We are still building the Course Advisor Agent.
*   The Goal: Answer "What ML courses are there?"
*   The Change: Instead of dumping raw JSON, we will carefully curate the text we send to the LLM.

Let's see how much better we can do.

# Setup and Initialization

Let's set up our environment and import the Stage 2 agent.

In [ ]:
import sys
import os
import asyncio
from pathlib import Path
from dotenv import load_dotenv

# 1. Configure Paths
# We point to 'stage2_context_engineered' this time
project_root = Path("../../").resolve()
stage2_path = project_root / "progressive_agents" / "stage2_context_engineered"
sys.path.append(str(stage2_path))

# Add src to path to import models
src_path = project_root / "src"
sys.path.append(str(src_path))

# 2. Load Environment Variables
load_dotenv(project_root / ".env")

print(f"Project Root: {project_root}")
print(f"Agent Path Added: {stage2_path}")
print(f"Src Path Added: {src_path}")

In [ ]:
from agent import setup_agent

print("Initializing Stage 2 Agent...")
# This reuses the same Redis data from Stage 1 (or generates it if missing)
workflow, course_manager = setup_agent(auto_load_courses=True)
print("Agent is ready!")

In [ ]:
import agent.context_engineering

# Inject our function into the agent's module
agent.context_engineering.transform_course_to_text = transform_course_to_text

print("Successfully injected your custom function into the agent!")

## Step 2: Integrate with the Agent

Now that we have our logic, let's inject it into the agent. We will "monkeypatch" the agent's module to use *our* function instead of the default one. This proves that your code is driving the agent!

In [ ]:
def transform_course_to_text(course: Course) -> str:
    """
    Transform course object to LLM-optimized text format.
    """
    # Build prerequisites text
    prereq_text = ""
    if course.prerequisites:
        prereq_codes = [p.course_code for p in course.prerequisites]
        prereq_text = f"\nPrerequisites: {', '.join(prereq_codes)}"

    # Build learning objectives text
    objectives_text = ""
    if course.learning_objectives:
        objectives_text = f"\nLearning Objectives:\n" + "\n".join(
            f"  - {obj}" for obj in course.learning_objectives
        )

    # Build course text (CLEANED and TRANSFORMED)
    course_text = f"""{course.course_code}: {course.title}
Department: {course.department}
Credits: {course.credits}
Level: {course.difficulty_level.value}
Format: {course.format.value}
Instructor: {course.instructor}{prereq_text}
Description: {course.description}{objectives_text}"""

    return course_text

# Test it!
print("--- Engineered Context (Clean Text) ---")
print(transform_course_to_text(sample_course))

### Exercise: Implement Transformation

Now, write a function that converts this object into a clean, readable string.
*   Include: Code, Title, Department, Credits, Instructor, Description.
*   Exclude: ID, timestamps, enrollment numbers.
*   Format: Key: Value (Natural Text).

In [ ]:
from redis_context_course.models import Course, DifficultyLevel, CourseFormat

# Create a dummy course for testing
sample_course = Course(
    id="course_12345",
    course_code="CS101",
    title="Introduction to Computer Science",
    department="Computer Science",
    credits=4,
    difficulty_level=DifficultyLevel.BEGINNER,
    format=CourseFormat.IN_PERSON,
    instructor="Dr. Alice Smith",
    description="A fundamental course covering the basics of programming and algorithms.",
    prerequisites=[],
    learning_objectives=["Understand variables", "Write loops"],
    # Noise fields that we want to remove:
    created_at="2023-01-01T00:00:00Z",
    updated_at="2023-06-01T00:00:00Z",
    enrollment_capacity=100,
    current_enrollment=85
)

print("--- Raw Course Data (JSON) ---")
print(sample_course.model_dump_json(indent=2))

## Step 1: Define the Context Engineering Logic

Before we run the agent, let's define the logic that will clean and transform our data.

We will implement a function `transform_course_to_text` that takes a `Course` object and returns a clean string.

First, let's look at what a "Raw" course looks like.

## The Experiment: "What ML courses are available?"

We will run the **exact same query** as in Stage 1. This allows us to make a direct comparison.

> *"What machine learning courses are available?"*

In [ ]:
# Define the user's query
query = "What machine learning courses are available?"

print(f"User asks: '{query}'")
print("Running workflow...")

# Run the graph
result = await workflow.ainvoke({"query": query})

print("Workflow complete!")

## Analysis: The Power of Context Engineering

Let's look at the results. In Stage 1, this query cost us ~6,000 tokens. How did we do this time?

In [ ]:
# Display the Answer
print("="*60)
print(f"Agent Answer:\n\n{result['final_answer']}")
print("="*60)

# Display the Metrics
courses_found = result.get('courses_found', 0)
total_tokens = result.get('total_tokens', 0)

print(f"\nStatistics:")
print(f"   Courses Retrieved: {courses_found}")
print(f"   Total Tokens Used: {total_tokens:,}")

### The Comparison

| Metric | Stage 1 (Baseline) | Stage 2 (Engineered) | Improvement |
| :--- | :--- | :--- | :--- |
| **Total Tokens** | ~6,100 | **~540** | **~91% Reduction** |
| **Format** | Raw JSON | Clean Text | Better Readability |
| **Noise** | High (Syllabi, IDs) | Low (Relevant info only) | Focused Context |

We achieved a **91% reduction in token usage** simply by cleaning and formatting our data!

### Inspecting the Engineered Context
Let's see what the LLM actually saw this time.

In [ ]:
# Inspect the engineered context
engineered_context = result.get('engineered_context', '')

print(f"Total Context Size: {len(engineered_context):,} characters")
print("-" * 40)
print("PREVIEW OF ENGINEERED CONTEXT")
print("-" * 40)
print(engineered_context)

## Conclusion

Context Engineering is a powerful tool for optimization. By curating our data, we saved money and improved speed.

Key Takeaways:
1. Clean your data: Remove database artifacts.
2. Transform your data: Use formats that are token-efficient.
3. Optimize for the task: Summaries are great for search, but maybe not for deep dives.

### Next Lesson: Hierarchical Retrieval
In **Stage 3**, we will solve the "Flat Retrieval" problem. We will build a "Smart" agent that:
1. Retrieves summaries first (like Stage 2).
2. Decides which course is most relevant.
3. Fetches the **full syllabus** for *only* that course.

This gives us the best of both worlds: Low token usage *and* high detail when needed.

See you in Stage 3!